# Итак, давайте построим RAG систему по информационной безопасности

## Сперва необходимо собрать документы для базы знаний
Я выбрал 10 распространённых тем по информационной безопасности. Для каждой темы нашёл наиболее релевантный по содержанию документ, как правило это информация с сайтов, статей, whitepaper'ов на русском и английском языке. Я перевёл их на русский язык и чуть отформатировал текст.

Все исходники можете найти на диске - https://drive.google.com/drive/folders/1z4KwAa21gzyVY2KGkbv4houCNpFk3nzl

Давайте скачаем документы

In [1]:
!pip install gdown
!pip install auto-gptq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires 

In [2]:
import gdown
import zipfile
import os
import shutil

# Ссылка на скачивание
url = 'https://drive.google.com/uc?id=16ZmD132N5k0g8utbpPn4pbiRAO_W3wbi'
output = 'knowledge_base.zip'

gdown.download(url, output, quiet=False)

temp_dir = 'temp_knowledge_base'
if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)

with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall(temp_dir)

print("Файлы во временной папке:", os.listdir(temp_dir))

if not os.path.exists('knowledge_base'):
    os.makedirs('knowledge_base')

for file_name in os.listdir(temp_dir):
    shutil.move(os.path.join(temp_dir, file_name), 'knowledge_base')

shutil.rmtree(temp_dir)

print("Файлы в папке 'knowledge_base':", os.listdir('knowledge_base'))

Downloading...
From: https://drive.google.com/uc?id=16ZmD132N5k0g8utbpPn4pbiRAO_W3wbi
To: /content/knowledge_base.zip
100%|██████████| 68.9k/68.9k [00:00<00:00, 53.5MB/s]

Файлы во временной папке: ['iot.txt', 'AI.txt', 'ics.txt', 'audit.txt', 'fishing.txt', 'ransomware.txt', 'cloud.txt', 'ddos.txt', 'remote.txt']
Файлы в папке 'knowledge_base': ['iot.txt', 'AI.txt', 'ics.txt', 'audit.txt', 'fishing.txt', 'ransomware.txt', 'cloud.txt', 'ddos.txt', 'remote.txt']


Хоть текст уже предобработан, на всякий случай давайте ещё чуть отформатируем его

In [3]:
!pip install -q langchain
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 176.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.5 MB/s eta 0:00:00


In [4]:
import re
from langchain.document_loaders import TextLoader

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

directory = "knowledge_base"
all_documents = []

if not os.path.exists(directory) or not os.listdir(directory):
    raise Exception(f"Directory {directory} is empty or doesn't exist.")

for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        filepath = os.path.join(directory, filename)
        loader = TextLoader(filepath, encoding='utf-8')
        documents = loader.load()
        all_documents.extend(documents)

for doc in all_documents:
    doc.page_content = clean_text(doc.page_content)

## Добавление документов в FAISS

Теперь нужно каждый документ разбить на чанки. Возьмём за стандарт размер чанка 1000 с перекрытием 150

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Используем разбиение с учётом границ предложений и абзацев
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    separators=["\n\n", ".", "!", "?"]
)

docs = text_splitter.split_documents(all_documents)

Теперь давайте сделаем эмбединги для чанков и добаим их в хранилище FAISS

In [6]:
!pip install -q sentence-transformers
!pip install -q faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.0 MB/s eta 0:00:00


In [7]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

docs_as_strings = [doc.page_content for doc in docs]

modelPath = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

embeddings = HuggingFaceEmbeddings(model_name=modelPath, encode_kwargs={'normalize_embeddings': True})

faiss_store = FAISS.from_texts(docs_as_strings, embeddings)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
<ipython-input-7-adbd1f7a9b5a>:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=modelPath, encode_kwargs={'normalize_embeddings': True})
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Us

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Нам необходимо выбрать retriever. Выбирать какой-то конкретный, как правило, решение не лучшее, поэтому давайте создадим ансамбль из двух типов: основанном на KNN и основанном на TF-IDF

In [8]:
!pip install rank_bm25

In [9]:
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers.bm25 import BM25Retriever

bm25_retriever = BM25Retriever.from_texts(docs_as_strings)

# В общем случае скажем, что веса ретриеверов будут равнозначными
ensemble_retriever = EnsembleRetriever(
    retrievers=[faiss_store.as_retriever(), bm25_retriever],
    weights=[0.5, 0.5]
)

Сформулируем запрос

In [10]:
question = "Вопрос: Как обеспечить безопасность удаленной работы сотрудников?"

Уже на этом этапе мы можем найти по нашему запросу похожие чанки из нашей базы знаний. Похожие чанки нужно отранжировать, сделаем это с помощью CrossEncoder'а

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

cross_encoder_model = AutoModelForSequenceClassification.from_pretrained("cross-encoder/ms-marco-MiniLM-L-6-v2")
cross_encoder_tokenizer = AutoTokenizer.from_pretrained("cross-encoder/ms-marco-MiniLM-L-6-v2")

def cross_encoder_rerank(query, documents):
    inputs = cross_encoder_tokenizer([query] * len(documents), documents, return_tensors="pt", padding=True, truncation=True)
    scores = cross_encoder_model(**inputs).logits
    ranked_docs = sorted(zip(documents, scores.squeeze().tolist()), key=lambda x: x[1], reverse=True)
    return [doc for doc, score in ranked_docs]

def get_context_from_retriever(query):
    search_results = ensemble_retriever.get_relevant_documents(query)
    search_results_as_strings = [doc.page_content for doc in search_results]
    ranked_results = cross_encoder_rerank(query, search_results_as_strings)
    return ranked_results

retrieved_docs = get_context_from_retriever(question)

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
<ipython-input-11-0900dd4749e3>:13: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  search_results = ensemble_retriever.get_relevant_documents(query)


Теперь среди чанков, которые мы получили из базы знаний, постраемся найти дейтвительно релевантные запросу. Значения трешхолдов подобраны вручную на основе тестовых запросов.

In [12]:
!pip install pymorphy2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 50.4 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=9c35de458498ef777e6db8cd3f74a498352aa3b4b4fede48750c211c719271bc
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [13]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

В моём случае лучше всего было проверять релевантность на основе TF-IDF. Так как у нас рускоязычный текст, то при проверке приведём слова в начальную форму (лемматизируем), чтобы лучше находить общие слова

In [14]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import pymorphy2

# Инициализация лемматизатора, основанного на словарях OpenCorpora
morph = pymorphy2.MorphAnalyzer()

def lemmatize_text(text):
    words = text.split()
    lemmatized_words = [morph.parse(word)[0].normal_form for word in words]
    return " ".join(lemmatized_words)

def is_relevant_chunk(question, chunk, threshold=0.15):
    question_lemmatized = lemmatize_text(question)
    chunk_lemmatized = lemmatize_text(chunk)

    if not question_lemmatized or not chunk_lemmatized:
        return False

    vectorizer = TfidfVectorizer()

    tfidf_matrix = vectorizer.fit_transform([question_lemmatized, chunk_lemmatized])

    question_tfidf = tfidf_matrix[0].toarray()[0]
    chunk_tfidf = tfidf_matrix[1].toarray()[0]

    # Вычисляем схожесть как сумму минимальных пересекающихся TF-IDF значений
    similarity_score = sum(min(q, c) for q, c in zip(question_tfidf, chunk_tfidf))

    return similarity_score >= threshold

def preprocess(text):
    russian_stopwords = set(stopwords.words('russian'))
    punctuation_set = set(string.punctuation)
    text = text.lower()
    text = re.sub(f"[{re.escape(''.join(punctuation_set))}]", " ", text)

    tokens = word_tokenize(text, language='russian')

    # Удаление стоп-слов и неалфавитных токенов
    filtered_tokens = [word for word in tokens if word.isalpha() and word not in russian_stopwords]

    return ' '.join(filtered_tokens)

def get_relevant_docs(question, retrieved_docs):
    relevant_docs = []
    for doc in retrieved_docs:
        if is_relevant_chunk(question, doc):
            relevant_docs.append(doc)
    return relevant_docs

relevant_docs = get_relevant_docs(question, retrieved_docs)

Соотвественно, контекстом будут являться все релевантные чанки с учётом порядка

In [15]:
context = " ".join(relevant_docs)

In [16]:
print(context)

. д. Но нельзя забывать, что злоумышленники могут получить доступ к веб-камере и нарушить неприкосновенность вашей личной жизни. А если на рабочем месте лежат конфиденциальные документы, хакеры могут просматривать их через веб-камеру. Если у вас внешняя камера, отключайте ее, когда не используете. Если же камера встроена, лучше дополнительно защитить ее – предугадать атаку, увы, нельзя. В интернет-магазинах продаются крышки для веб-камер любых форм, размеров и цветов. Установить такую крышку очень просто: обычно она крепится специальным клейким слоем. Некоторые платформы для телеконференций поддерживают функцию размытия фона и подобные. Они скрывают от других участников обстановку вокруг, а значит – и конфиденциальную информацию о вас и ваших клиентах. 4. Обеспечьте максимальную защиту корпоративной сети VPN В условиях удаленной работы к виртуальным частным сетям компаний (Virtual Private Network, VPN) подключено больше пользователей, чем обычно Удаленная работа: 10 советов по онлайн-б

In [17]:
context = context[5:]

print(context)

Но нельзя забывать, что злоумышленники могут получить доступ к веб-камере и нарушить неприкосновенность вашей личной жизни. А если на рабочем месте лежат конфиденциальные документы, хакеры могут просматривать их через веб-камеру. Если у вас внешняя камера, отключайте ее, когда не используете. Если же камера встроена, лучше дополнительно защитить ее – предугадать атаку, увы, нельзя. В интернет-магазинах продаются крышки для веб-камер любых форм, размеров и цветов. Установить такую крышку очень просто: обычно она крепится специальным клейким слоем. Некоторые платформы для телеконференций поддерживают функцию размытия фона и подобные. Они скрывают от других участников обстановку вокруг, а значит – и конфиденциальную информацию о вас и ваших клиентах. 4. Обеспечьте максимальную защиту корпоративной сети VPN В условиях удаленной работы к виртуальным частным сетям компаний (Virtual Private Network, VPN) подключено больше пользователей, чем обычно Удаленная работа: 10 советов по онлайн-безопа

Давайте посмотрим на метрики релевантных чанков, чтобы понять, если ли у нас релевантная информация в базе знаний по теме нашего запроса.

In [18]:
# Метрика точности на K элементах
def precision_at_k(relevant_docs, retrieved_docs, k):
    k = min(k, len(retrieved_docs))
    relevant_at_k = [doc in relevant_docs for doc in retrieved_docs[:k]]
    return sum(relevant_at_k) / k

# Метрика, которая учитывает порядок чанков
def average_precision_at_k(relevant_docs, retrieved_docs, k):
    k = min(k, len(retrieved_docs))
    score = 0.0
    relevant_count = 0
    for i in range(1, k+1):
        if retrieved_docs[i-1] in relevant_docs:
            relevant_count += 1
            score += relevant_count / i
    return score / len(relevant_docs) if relevant_docs else 0

# Метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции
def dcg_at_k(relevant_docs, retrieved_docs, k):
    k = min(k, len(retrieved_docs))
    dcg = 0.0
    for i in range(1, k+1):
        if retrieved_docs[i-1] in relevant_docs:
            dcg += 1 / np.log2(i + 1)
    return dcg

# Нормализованная версия предыдущей метрики
def ndcg_at_k(relevant_docs, retrieved_docs, k):
    k = min(k, len(retrieved_docs))  # Ensure k does not exceed the length of retrieved_docs
    dcg_max = dcg_at_k(relevant_docs, relevant_docs[:k], k)
    if not dcg_max:
        return 0.0
    return dcg_at_k(relevant_docs, retrieved_docs, k) / dcg_max

In [19]:
import numpy as np

k = len(relevant_docs)

p_at_k = precision_at_k(relevant_docs, retrieved_docs, k)
map_at_k = average_precision_at_k(relevant_docs, retrieved_docs, k)
ndcg_at_k_value = ndcg_at_k(relevant_docs, retrieved_docs, k)

print(f"Precision@K: {p_at_k}, MAP@K: {map_at_k}, NDCG@K: {ndcg_at_k_value}")

Precision@K: 0.5, MAP@K: 0.20833333333333331, NDCG@K: 0.36331756126716835


Эти метрики будут учитываться при формировании ответа

## Получение базового ответа LLM

Я выбрал русскоязычную LLM, так как её можно легко развернуть в колабе, она достаточно хороша, она умещается на T4 и её инференс занимает умеренное количество времени.

Давайте определимся с гиперпараметрами модели, сделаем небольшой промпт инженеринг, используя технику few-shot, чтобы помочь нашей модели.

In [20]:
from auto_gptq import AutoGPTQForCausalLM
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline

# Инициализация ruGPT от DeepPavlov
model_name = 'fffrrt/ruGPT-3.5-13B-GPTQ'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name,
    model_basename='gptq_model-4bit-128g',
    use_safetensors=True,
    trust_remote_code=True,
    device="cuda:0"
)

# Выбрал режим генерации текста, подобрал гиперпараметры, чтобы уменьшить риск галлюцинации
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    temperature=0.2,
    max_new_tokens=50,
    repetition_penalty=1.2,
    top_p=0.9,
    top_k=40,
    do_sample=True
)

llm = HuggingFacePipeline(pipeline=generator)

# Few-shot
few_shot_examples = """
Пример 1:
Вопрос: Что такое фишинг?
Ответ: Фишинг - это вид кибератаки, при которой злоумышленники пытаются обманным путем получить личные данные.

Пример 2:
Вопрос: Что такое шифрование данных?
Ответ: Шифрование данных - это процесс преобразования информации в формат, который могут читать только авторизованные пользователи.

Пример 3:
Вопрос: Как работает антивирусное ПО?
Ответ: Антивирусное программное обеспечение сканирует файлы и программы на наличие вредоносного кода.
"""

# Сделаем небольшой промпт инженеринг для первоначального ответа
primary_prompt_template = few_shot_examples + """
Вопрос: {query}
Ответ: """

primary_prompt = PromptTemplate(
    input_variables=["query"],
    template=primary_prompt_template,
)

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.77M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/582 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/816 [00:00<?, ?B/s]

WARNING - Exllamav2 kernel is not installed, reset disable_exllamav2 to True. This may because you installed auto_gptq using a pre-build wheel on Windows, in which exllama_kernels are not compiled. To use exllama_kernels to further speedup inference, you can re-install auto_gptq from source.
WARNING - CUDA kernels for auto_gptq are not installed, this will result in very slow inference speed. This may because:
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads 

quantize_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

gptq_model-4bit-128g.safetensors:   0%|          | 0.00/7.60G [00:00<?, ?B/s]

INFO - The layer lm_head is not quantized.
INFO:auto_gptq.modeling._base:The layer lm_head is not quantized.
The model 'GPT2GPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM

Сгенерируем ответ на наш вопрос. Возьмём только первое предложение, так как эта модель, как правило весь смысл выдаёт в первом предложении, затем велик риск галюцинации.

In [21]:
def generate_general_answer(question):
    prompt = primary_prompt.format(query=question)
    result = generator(prompt, return_full_text=False)
    answer = result[0]['generated_text'].replace(prompt, '').strip()
    return answer

def get_first_sentence(text):
    sentences = re.split(r'(?<=[.!?])\s+(?=[А-ЯA-Z])', text)
    return sentences[0]

general_answer = generate_general_answer(question)
general_answer = get_first_sentence(general_answer)

In [22]:
general_answer

'В первую очередь необходимо убедиться, что сотрудники не имеют доступа к компьютеру с установленным приложением TeamViewer или другим подобным программным обеспечением для удаленного управления компьютером (например, Remote Desktop).'

Сгенерировав первоначальный ответ, мы также можем оценить его релевантность запросу.

In [23]:
lemmatized_question = lemmatize_text(question)
lemmatized_answer = lemmatize_text(general_answer)

vectorizer = TfidfVectorizer()

tfidf_matrix = vectorizer.fit_transform([lemmatized_question, lemmatized_answer])

# Получаем словарь TF-IDF значений для каждого документа
feature_names = vectorizer.get_feature_names_out()

# Извлекаем TF-IDF веса для вопроса и ответа
question_tfidf = tfidf_matrix[0].toarray()[0]
answer_tfidf = tfidf_matrix[1].toarray()[0]

# Вычисляем схожесть как сумму пересекающихся TF-IDF значений
similarity_score = 0
for i, term in enumerate(feature_names):
    if question_tfidf[i] > 0 and answer_tfidf[i] > 0:
        similarity_score += min(question_tfidf[i], answer_tfidf[i])  # Можно брать минимум или сумму

print(f"TF-IDF based similarity score: {similarity_score}")

TF-IDF based similarity score: 0.13819929773426579


In [24]:
print('Question: ', question)
print('General answer: ', general_answer)

Question:  Вопрос: Как обеспечить безопасность удаленной работы сотрудников?
General answer:  В первую очередь необходимо убедиться, что сотрудники не имеют доступа к компьютеру с установленным приложением TeamViewer или другим подобным программным обеспечением для удаленного управления компьютером (например, Remote Desktop).


## Получение уточнённого ответа

Давайте определим, на какие метрики мы можем полагаться

In [25]:
precision_threshold = 0.15
tfidf_threshold = 0.1

def is_metrics_good(precision, tfidf_score):
    quality_metrics_good = precision >= precision_threshold
    tfidf_good = tfidf_score >= tfidf_threshold

    return quality_metrics_good, tfidf_good

quality_metrics_good, tfidf_good = is_metrics_good(
    p_at_k, similarity_score
)

В зависимости от качества метрик, сформируем уточнённый ответ

In [26]:
import re

# Учитывая возможности нашей LLM, ограничим длину контекста
def truncate_to_max_length(text):
    sentences = re.split(r'(?<=[.!?])\s+', text)
    truncated_text = ' '.join(sentences[:1])
    return truncated_text

# Логика для генерации уточненного ответа
if not quality_metrics_good and not tfidf_good:
    # Все метрики плохие
    refined_answer = "Мы не знаем ответа на вопрос."
elif quality_metrics_good and not tfidf_good:
    # Первоначальный контекст, передаём только контекст
    rephrased_context = truncate_to_max_length(context)
    secondary_prompt = PromptTemplate(
        input_variables=["context", "question"],
        template="Сформулируй ответ на вопрос {question} на основе информации {context}",
    )
    prompt = secondary_prompt.format(context=rephrased_context, question=question)
    result = generator(prompt, return_full_text=False)
    refined_answer = result[0]['generated_text'].replace(prompt, '').strip()
elif not quality_metrics_good and tfidf_good:
    # Контекст плохой, возращаем первоначальный ответ
    refined_answer = general_answer
else:
    # Все метрики хорошие, передаём и контекст, и общий ответ
    rephrased_context = truncate_to_max_length(context)
    secondary_prompt = PromptTemplate(
        input_variables=["context", "general_answer", "question"],
        template="Уточни ответ {general_answer} на вопрос {question} на основе информации {context}",
    )
    prompt = secondary_prompt.format(context=rephrased_context, question=question, general_answer=general_answer)
    result = generator(prompt, return_full_text=False)
    refined_answer = result[0]['generated_text'].replace(prompt, '').strip()

Итоговый ответ

In [27]:
refined_answer

'На этом этапе нужно определить, какие действия будут выполняться при запуске программы TeamViewer. Если вы используете программу в первый раз, то вам следует ознакомиться со следующими разделами этой статьи. \n Для того чтобы запустить приложение, нажмите кнопку "Пуск'

P.S. Итого, мы построили RAG систему. (Если бы у меня было больше времени, то я бы не побоялся проблем с оплатой и использовал бы OpenAI Api и получил бы гораздо более качественнные ответы).  Полагаю, вышеописанный код и текстовые комментарии позволили понять мой ход мыслей. Буду ждать обратную связь.